# MCP-MD: Google Colab Setup

This notebook prepares Google Colab environment for running MD simulations with the MCP-MD workflow.

## Prerequisites

1. **Runtime Type**: Go to `Runtime > Change runtime type > GPU` for faster OpenMM simulations
2. **Project Files**: Upload your project files to Google Drive or clone from GitHub

## Setup Steps

1. **Cell 1**: Install condacolab (triggers runtime restart)
2. **Cell 2**: Install AmberTools and Python packages (run after restart)
3. **Cell 3**: Get project files (Google Drive or GitHub)
4. **Cell 4**: Verify all dependencies

⚠️ **Important**: After Cell 1, the runtime will restart automatically. Run Cell 2 after restart.


---
## Cell 1: Install condacolab

This cell installs condacolab which enables conda in Google Colab.

**⚠️ The runtime will restart after this cell. This is expected!**


In [ ]:
# ========================================
# Cell 1: Install condacolab
# ========================================
# This triggers a runtime restart. Run Cell 2 after restart.

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🔧 Installing condacolab...")
    print("⚠️  The runtime will restart after installation.")
    print("   After restart, run Cell 2.")
    print()
    !pip install -q condacolab
    import condacolab
    condacolab.install()
else:
    print("✅ Not running in Colab - skipping condacolab setup")
    print("   Make sure you have conda/mamba installed locally.")


---
## Cell 2: Install Dependencies

**Run this cell AFTER the runtime restarts from Cell 1.**

This installs:
- **AmberTools** (antechamber, tleap, pdb4amber, packmol-memgen) via conda
- **Python packages** (gemmi, rdkit, pdbfixer, openmm, py3Dmol, mdtraj, dimorphite_dl) via pip

⏱️ This takes approximately **5-10 minutes**.


In [ ]:
# ========================================
# Cell 2: Install Dependencies
# ========================================
# Run this AFTER runtime restart from Cell 1

import sys
import time

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # Verify condacolab is working
    import condacolab
    condacolab.check()
    
    start_time = time.time()
    
    # ===== AmberTools via conda =====
    print("="*60)
    print("📦 Installing AmberTools via conda...")
    print("   (This takes ~5-10 minutes)")
    print("="*60)
    !conda install -y -c conda-forge ambertools=23 2>&1 | tail -20
    print(f"✅ AmberTools installed ({time.time() - start_time:.0f}s)")
    
    # ===== Python packages via pip =====
    print()
    print("="*60)
    print("📦 Installing Python packages via pip...")
    print("="*60)
    
    # Core packages
    !pip install -q gemmi rdkit-pypi pdbfixer openmm py3Dmol mdtraj
    print("✅ Core packages installed")
    
    # dimorphite_dl (for pH-dependent protonation)
    !pip install -q git+https://github.com/durrantlab/dimorphite_dl.git
    print("✅ dimorphite_dl installed")
    
    total_time = time.time() - start_time
    print()
    print("="*60)
    print(f"✅ All dependencies installed! (Total: {total_time/60:.1f} minutes)")
    print("="*60)
    
else:
    print("✅ Not running in Colab")
    print("   Ensure you have the following installed:")
    print("   - AmberTools (conda install -c conda-forge ambertools)")
    print("   - gemmi, rdkit, pdbfixer, openmm, py3Dmol, mdtraj, dimorphite_dl")


---
## Cell 3: Get Project Files

Choose one of the following options:

### Option A: Clone from GitHub (if public repository)
Uncomment the GitHub clone lines.

### Option B: Mount Google Drive
1. Upload the `mcp-md` folder to your Google Drive
2. Run this cell to mount Drive and copy files


In [ ]:
# ========================================
# Cell 3: Get Project Files
# ========================================

import sys
import os

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # ===== Option A: Clone from GitHub =====
    # Uncomment the following lines if your repo is public:
    
    # GITHUB_REPO = "https://github.com/YOUR_USERNAME/mcp-md.git"
    # !git clone {GITHUB_REPO} /content/mcp-md
    # %cd /content/mcp-md/notebooks
    # sys.path.insert(0, '/content/mcp-md')
    
    # ===== Option B: Mount Google Drive =====
    # Assumes you have uploaded mcp-md folder to: My Drive/mcp-md/
    
    print("📂 Mounting Google Drive...")
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Path to your project in Google Drive
    DRIVE_PATH = "/content/drive/MyDrive/mcp-md"
    
    if os.path.exists(DRIVE_PATH):
        print(f"✅ Found project at: {DRIVE_PATH}")
        
        # Copy to /content for faster access
        print("📋 Copying files to /content/mcp-md ...")
        !cp -r "{DRIVE_PATH}" /content/
        
        %cd /content/mcp-md/notebooks
        sys.path.insert(0, '/content/mcp-md')
        
        print(f"✅ Working directory: {os.getcwd()}")
    else:
        print(f"❌ Project not found at: {DRIVE_PATH}")
        print("   Please upload your mcp-md folder to Google Drive.")
        print("   Expected location: My Drive/mcp-md/")
        
else:
    # Local development - just add parent to path
    sys.path.insert(0, '..')
    print(f"✅ Working directory: {os.getcwd()}")
    print("   Parent directory added to sys.path")


---
## Cell 4: Verify Dependencies

This cell checks that all required packages and tools are installed correctly.


In [ ]:
# ========================================
# Cell 4: Verify Dependencies
# ========================================

import shutil

print("="*60)
print("🔍 Dependency Verification")
print("="*60)

all_ok = True

# ===== Python Packages =====
print("\n📦 Python Packages:")
packages = [
    ("gemmi", "gemmi"),
    ("rdkit", "rdkit"),
    ("dimorphite_dl", "dimorphite_dl"),
    ("pdbfixer", "pdbfixer"),
    ("openmm", "openmm"),
    ("py3Dmol", "py3Dmol"),
    ("mdtraj", "mdtraj"),
    ("numpy", "numpy"),
]

for display_name, import_name in packages:
    try:
        __import__(import_name)
        print(f"  ✅ {display_name}")
    except ImportError as e:
        print(f"  ❌ {display_name}: {e}")
        all_ok = False

# ===== AmberTools =====
print("\n🔧 AmberTools:")
tools = [
    "antechamber",
    "parmchk2",
    "pdb4amber",
    "tleap",
    "packmol-memgen",
]

for tool in tools:
    path = shutil.which(tool)
    if path:
        print(f"  ✅ {tool}")
    else:
        print(f"  ❌ {tool}: not found in PATH")
        all_ok = False

# ===== Custom Modules =====
print("\n📁 Custom Modules:")
modules = [
    ("servers.structure_server", "Structure Server"),
    ("servers.solvation_server", "Solvation Server"),
    ("servers.amber_server", "Amber Server"),
    ("common.base", "Common Base"),
]

for module_name, display_name in modules:
    try:
        __import__(module_name)
        print(f"  ✅ {display_name}")
    except ImportError as e:
        print(f"  ❌ {display_name}: {e}")
        all_ok = False

# ===== OpenMM Platform =====
print("\n🖥️ OpenMM Platforms:")
try:
    import openmm as mm
    for i in range(mm.Platform.getNumPlatforms()):
        platform = mm.Platform.getPlatform(i)
        name = platform.getName()
        if name == 'CUDA':
            print(f"  ✅ {name} (GPU - fastest)")
        elif name == 'OpenCL':
            print(f"  ✅ {name} (GPU)")
        elif name == 'CPU':
            print(f"  ✅ {name}")
        else:
            print(f"  ✅ {name}")
except Exception as e:
    print(f"  ❌ Could not check platforms: {e}")
    all_ok = False

# ===== Summary =====
print("\n" + "="*60)
if all_ok:
    print("✅ All dependencies verified successfully!")
    print("   You can now run test_amber_boltz.ipynb or test_amber_general.ipynb")
else:
    print("⚠️  Some dependencies are missing.")
    print("   Please check the errors above and re-run the setup cells.")
print("="*60)


---
## Next Steps

Once all dependencies are verified, you can:

1. **Run `test_amber_general.ipynb`**: MD workflow with PDB structures (e.g., 1AKE)
2. **Run `test_amber_boltz.ipynb`**: MD workflow with Boltz-2 predicted structures

### Tips for Colab

- **GPU Runtime**: Change to GPU runtime for faster OpenMM simulations
- **Session Timeout**: Colab sessions timeout after ~90 minutes of inactivity
- **Save Results**: Download or save results to Google Drive before session ends
- **Re-run Setup**: If runtime resets, you need to re-run Cells 2-4


In [ ]:
# ========================================
# Quick Test: Structure Server
# ========================================
# Run this to verify the workflow is ready

print("🧪 Quick Test: Fetching PDB structure...")

try:
    import servers.structure_server as structure_server
    
    # Test fetch_molecules
    result = await structure_server.fetch_molecules(
        pdb_id="1CRN",  # Small protein (crambin)
        source="pdb",
        prefer_format="pdb"
    )
    
    if result["success"]:
        print(f"✅ Successfully fetched 1CRN")
        print(f"   File: {result['file_path']}")
        print(f"   Atoms: {result['num_atoms']}")
        print(f"   Chains: {result['chains']}")
        print("\n🎉 Setup complete! You're ready to run MD simulations.")
    else:
        print(f"❌ Failed: {result['errors']}")
        
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure you've run Cell 3 to get project files.")
